In [1]:
import argparse
import pandas as pd

import numpy as np
import torch
from torch.autograd import Variable

import data
import utils
import models
from domain import get_domain
from engines.engine import Criterion


In [2]:
parser = argparse.ArgumentParser(description='testing script')
parser.add_argument('--data', type=str, default='data/negotiate', help='location of the data corpus')
parser.add_argument('--unk_threshold', type=int, default=20, help='minimum word frequency to be in dictionary')
parser.add_argument('--model_file', type=str, help='pretrained model file')
parser.add_argument('--seed', type=int, default=1, help='random seed')
parser.add_argument('--hierarchical', action='store_true', default=False, help='use hierarchical model')
parser.add_argument('--bsz', type=int, default=16, help='batch size')
parser.add_argument('--cuda', action='store_true', default=False, help='use CUDA')
parser.add_argument('--domain', type=str, default='object_division', help='domain for the dialogue')
parser.add_argument('--sep_sel', action='store_true', default=False, help='use separate classifiers for selection')
parser.add_argument('--model_type', type=str, default='rnn_model', help='model type', choices=models.get_model_names())
parser.add_argument('--lr', type=float, default=20.0, help='initial learning rate')
parser.add_argument('--min_lr', type=float, default=1e-5, help='min threshold for learning rate annealing')
parser.add_argument('--decay_rate', type=float,  default=9.0, help='decrease learning rate by this factor')
parser.add_argument('--decay_every', type=int,  default=1, help='decrease learning rate after decay_every epochs')
parser.add_argument('--momentum', type=float, default=0.0, help='momentum for sgd')
parser.add_argument('--clip', type=float, default=0.2, help='gradient clipping')
parser.add_argument('--visual', action='store_true', default=False, help='plot graphs')
parser.add_argument('--sample_file', type=str, help='pretrained model file')
parser.add_argument('--prediction_model_file', type=str,  default='', help='path to save the prediction model')
parser.add_argument('--selection_model_file', type=str,  default='', help='path to save the selection model')
    


_StoreAction(option_strings=['--selection_model_file'], dest='selection_model_file', nargs=None, const=None, default='', type=<class 'str'>, choices=None, help='path to save the selection model', metavar=None)

In [3]:
args = parser.parse_args(['--cuda', '--model_file=clustering_model.th', '--bsz=1', '--domain=object_division', '--sep_sel', '--model_type=latent_clustering_model', '--lr=0.001', '--selection_model_file=selection_model.th'])

In [4]:
device_id = utils.use_cuda(args.cuda)
utils.set_seed(args.seed)

domain = get_domain(args.domain)
model_ty = models.get_model_type(args.model_type)

corpus = model_ty.corpus_ty(domain, args.data, freq_cutoff=args.unk_threshold,
        verbose=True, sep_sel=args.sep_sel)

model = utils.load_model(args.model_file)

crit = Criterion(model.word_dict, device_id=device_id)
sel_crit = Criterion(model.item_dict, device_id=device_id,
        bad_toks=['<disconnect>', '<disagree>'])

engine = model_ty.engine_ty(model, args, verbose=True)



testset, testset_stats = corpus.test_dataset(args.bsz)

total_valid_loss, total_select_loss, total_partner_ctx_loss, extra = engine.test_pass(testset, testset_stats)



dataset data/negotiate/train.txt, total 687919, unks 8718, ratio 1.27%
sample input i2w:  ['2', '1', '1', '6', '2', '1']
sample words:  ['YOU:', "i'd", 'like', 'the', 'hat', 'and', '1', 'ball', '.', '<eos>', 'THEM:', 'i', 'need', 'both', 'balls', '<eos>', 'YOU:', 'ok', '.', '<eos>', 'THEM:', '<selection>']
Sample output:  ['item0=2', 'item1=1', 'item2=0', 'item0=0', 'item1=0', 'item2=2']
dataset data/negotiate/val.txt, total 74653, unks 914, ratio 1.22%
sample input i2w:  ['1', '10', '3', '0', '1', '0']
sample words:  ['YOU:', 'i', 'need', 'the', 'book', 'and', 'two', 'hats', '<eos>', 'THEM:', 'i', 'get', 'the', 'ball', 'and', '1', 'hat', '<eos>', 'YOU:', 'actually', 'i', 'just', 'need', 'the', 'book', ',', 'so', 'you', 'can', 'have', 'the', 'rest', 'of', 'it', '<eos>', 'THEM:', 'you', 'get', 'book', 'since', 'its', 'worth', '10', 'to', 'you', 'i', 'get', 'the', 'rest', '.', 'deal', '<eos>', 'YOU:', '<selection>']
Sample output:  ['item0=1', 'item1=0', 'item2=0', 'item0=0', 'item1=3', 

NameError: name 'test_loss' is not defined

In [ ]:
    def _forward(self, batch):
        ctx, _, inpts, lens, tgts, sel_tgt, rev_idxs, hid_idxs, cnt = batch
        ctx = Variable(ctx)
        cnt = Variable(cnt)
        inpts = [Variable(inpt) for inpt in inpts]
        tgts = [Variable(tgt) for tgt in tgts]
        rev_idxs = [Variable(idx) for idx in rev_idxs]
        hid_idxs = [Variable(idx) for idx in hid_idxs]

        losses, stats = self.model.forward(inpts, tgts, hid_idxs, ctx, cnt)

        return losses, stats, lens

In [ ]:
extra['output'][0].shape

In [ ]:
def input_map(batch, sep_sel=True):
    ctx, _, inpts, lens, _, sel_tgt, rev_idxs, hid_idxs, _ = batch
    ctx = Variable(ctx)
    inpts = [Variable(inpt) for inpt in inpts]
    rev_idxs = [Variable(idx) for idx in rev_idxs]
    hid_idxs = [Variable(idx) for idx in hid_idxs]
    if sep_sel:
        sel_tgt = Variable(sel_tgt)
    else:
        sel_tgt = [Variable(t) for t in sel_tgt]

        # remove YOU:/THEM: from the end
    return(inpts[:-1], lens[:-1], rev_idxs[:-1], hid_idxs[:-1], ctx, sel_tgt)

In [ ]:
sentences = []
target = []
output = []
for i in range(len(extra['input'])):
    in1, in2, in3, in4, in5, tgt = input_map(extra['input'][i])
    out1 = extra['output'][i] 
    dialog = ''
    for i in in1:
        dialog = dialog + ' '.join(corpus.word_dict.i2w(i)) + ' '
    sentences.append(dialog)
    target.append(corpus.item_dict_old.i2w(tgt))
    output.append(corpus.item_dict_old.i2w(torch.argmax(out1, dim=1)))

In [ ]:
df = pd.DataFrame({'sentences': sentences, 'targets':target, 'outputs':output})
df

In [ ]:
(df['targets'] == df['outputs']).mean()

In [ ]:
for i in in1:
    print(corpus.word_dict.i2w(i))

In [ ]:
in4

In [ ]:
corpus.word_dict.i2w(in1[0][:,0])

In [ ]:
corpus.context_dict.i2w(in5[:,0])

In [ ]:
corpus.word_dict.i2w(in1[0][:,2])

In [ ]:
corpus.item_dict_old.i2w(in5[:,0])

In [ ]:
#             batches.append((ctx, partner_ctx, inpts, lens, tgts, sel_tgt, rev_idxs, hid_idxs, cnt))

In [ ]:
def make_readable(corpus, batch):
    

In [ ]:
type(corpus)